<a href="https://colab.research.google.com/github/Hugo-Zh0/YoloV12-Object-Detection-Project/blob/main/YOLOv12_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🚀 YOLOv12 Object Detection Project

[![Python](https://img.shields.io/badge/Python-3.10%2B-blue.svg)](https://www.python.org/downloads/)
[![Anaconda](https://img.shields.io/badge/Anaconda-Navigator-green.svg)](https://www.anaconda.com/download)
[![VS Code](https://img.shields.io/badge/Editor-VS%20Code-blue.svg)](https://code.visualstudio.com/)
[![Ultralytics](https://img.shields.io/badge/YOLOv12-Ultralytics-yellow.svg)](https://github.com/ultralytics/ultralytics)
[![License](https://img.shields.io/badge/License-MIT-lightgrey.svg)](LICENSE)

---
<br>

## 📌 Overview
A **collaborative group project** by Swinburne University students in partnership with **CSIRO**.  
This repository contains the setup, configuration, and workflow for training and running **YOLOv12** object detection models.

**👨‍💻 Team Members:** Harron, Feng, Bunmi, Huss, Hugo.

---
**Repo:** `Hugo-Zh0/YoloV12-Object-Detection-Project`  
**What you’ll do:**
1. Check runtime & GPU
2. Install dependencies
3. Clone your repo
4. Set paths in repository
5. Train
6. Validate
7. Predict
8. Export
10. Troubleshoot

<br>

#### **What to do after completion or if you don't want to run anymore**

After completing this colab you will need to export the folders(step 9) which includes the repository and runs, as the runtime session will expire when you close the website **(meaning the folders gets deleted)**.

<br>

#### **Starting from previous session**

If you are rerunning this agin, you will need to manually upload the folders back into the google colab againn (it has to be zipped up first to be uploaded)
Then you will need to run script to extract the folders back to original state.

From there you can start from Step 1,2,5,6,7,8 (excludes 3-4 as no need to clone repository and setting paths again)

## **Step 1 — 🚀 Runtime & GPU check**

## **Prerequistes**
*   Change runtime type to T4-GPU
*   Change runtime to Python 3
*   Have your dataset already downloaded







In [ ]:

#@title Check Python, CUDA, and PyTorch (Checks if runtime is all correct)
import sys, platform
print("Python:", sys.version)
print("Platform:", platform.platform())
try:
    import torch
    print("PyTorch:", torch.__version__)
    print("CUDA available:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("CUDA device:", torch.cuda.get_device_name(0))
except Exception as e:
    print("PyTorch not installed yet (will install in next step).")


## Step 2 — ⬇️ Install dependencies

In [ ]:

#@title Install Ultralytics & helpers
!pip install ultralytics

import torch, cv2, ultralytics
print("Ultralytics:", ultralytics.__version__)
print("Torch:", torch.__version__, "| CUDA:", torch.cuda.is_available())
print("OpenCV:", cv2.__version__)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.3 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics: 8.3.186
Torch: 2.8.0+cu126 | CUDA: True
OpenCV: 4.12.0


## Step 3 —  🤖 Clone your repository

In [ ]:

#@title Clone repository from github
REPO_URL = "https://github.com/Hugo-Zh0/YoloV12-Object-Detection-Project"
REPO_DIR = "/content/YoloV12-Object-Detection-Project"

import shutil, os, pathlib
if os.path.isdir(REPO_DIR):
    shutil.rmtree(REPO_DIR)

!git clone -q {REPO_URL} {REPO_DIR}
print("Cloned into:", REPO_DIR)


# 3.1 - Creating folders
Next we will need to manually create two folders under our cloned repository
* datasets
* yaml

Datasets:
* Folder will store our annotated datasets from roboflow

Yaml:

* Folder stores our yaml file from our dataset




### 3.1 - Import Datset Zip File into Colab

In [ ]:
from google.colab import files
import zipfile

# Upload the ZIP file
uploaded = files.upload()

In [ ]:
#@title Unzip Dataset and save to dataset folder
!unzip -q /content/koala.zip -d /content/YoloV12-Object-Detection-Project/datasets/koala
#!unzip -q /content/kangaroo.zip -d /content/YoloV12-Object-Detection-Project/datasets/kangaroo

## Step 4 — 🎞️ Set model & data paths

**Step 4.1**

Manually move YAML File stored in extracted dataset folder to previously created YAML Folder.

**Step 4.2**

Update the YAML File with proper location paths for: train, val, test

Double click the yaml file and it will open on the side.

**Koala:**

* /content/YoloV12-Object-Detection-Project/datasets/koala/train/images
* /content/YoloV12-Object-Detection-Project/datasets/koala/valid/images
* /content/YoloV12-Object-Detection-Project/datasets/koala/test/images


**Kangaroo:**

* /content/YoloV12-Object-Detection-Project/datasets/kangaroo/train/images
* /content/YoloV12-Object-Detection-Project/datasets/kangaroo/valid/images
* /content/YoloV12-Object-Detection-Project/datasets/kangaroo/test/images




Finally save the file doinng Ctrl+S

## Step 5 — 💥 Train (set your parameters)

In [ ]:
from ultralytics import YOLO
import torch, os, sys, requests

# --- Direct paths (edit to your repo mount point in Colab) ---
MODELS_DIR  = "/content/YoloV12-Object-Detection-Project/models"
MODEL_PATH  = os.path.join(MODELS_DIR, "yolo12s.pt")
DATA_YAML   = "/content/YoloV12-Object-Detection-Project/yaml/data.yaml"
RUNS_DIR    = "/content/YoloV12-Object-Detection-Project/runs/completed-training"

print("Model path:", MODEL_PATH)
print("Data yaml :", DATA_YAML)

# Device selection
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Load your local weights explicitly
model = YOLO(MODEL_PATH)

# Train without fetching any other checkpoints
results = model.train(
    data=DATA_YAML,
    epochs=5,
    imgsz=512,
    batch=12,
    workers=0,
    device=device,
    pretrained=False,
    amp=False,
    project=RUNS_DIR,
    name="train"
)

print("Save dir:", results.save_dir)

## Step 6 — ✅ Validate best model

In [ ]:

#@title Validate best.pt
# from pathlib import Path
# from ultralytics import YOLO

# RUNS_DIR = Path("/content/runs/detect/completed-training/") #change the train folder to latest/best training
# assert RUNS_DIR.exists(), "No training runs found. Train first."

# latest = sorted(RUNS_DIR.glob("**/"), key=lambda p: p.stat().st_mtime)[-1]
# best = latest / "weights" / "best.pt"
# print("Using:", best)
# assert best.exists(), f"Missing best.pt in {latest}/weights"

# model = YOLO(str(best))
# metrics = model.val(data=str(DATA_YAML), imgsz=512, device="cuda" if __import__("torch").cuda.is_available() else "cpu")
# print(metrics)

# Validate ALL runs under completed-training/, pick best, copy+rename best.pt
from pathlib import Path
from ultralytics import YOLO
import torch, yaml, re, shutil

# --- Paths ---
DATA_YAML = "/content/YoloV12-Object-Detection-Project/yaml/data.yaml"
RUNS_ROOT = Path("/content/YoloV12-Object-Detection-Project/runs/completed-training")
DEST_DIR  = Path("/content/YoloV12-Object-Detection-Project/models")
IMG_SIZE  = 512
DEVICE    = "cuda" if torch.cuda.is_available() else "cpu"

assert RUNS_ROOT.exists(), f"Missing runs dir: {RUNS_ROOT}"

# Derive dataset slug (e.g., 'koala'/'kangaroo') from data.yaml -> train path
with open(DATA_YAML, "r") as f:
    data_cfg = yaml.safe_load(f)
train_path = Path(str(data_cfg.get("train", "")))
dataset_slug = (train_path.parent.parent.name or "datasets") if len(train_path.parts) >= 3 else "datasets"

# Collect run dirs that have weights/best.pt
run_dirs = [d for d in RUNS_ROOT.iterdir() if d.is_dir() and (d / "weights" / "best.pt").exists()]
assert run_dirs, f"No runs with weights/best.pt under {RUNS_ROOT}"

# Validate each
records = []
for run in sorted(run_dirs, key=lambda p: p.stat().st_mtime):
    best_path = run / "weights" / "best.pt"
    model = YOLO(str(best_path))
    metrics = model.val(data=DATA_YAML, imgsz=IMG_SIZE, device=DEVICE, verbose=False)
    mAP   = float(getattr(metrics.box, "map", 0.0))      # mAP@0.50:0.95
    mAP50 = float(getattr(metrics.box, "map50", 0.0))    # mAP@0.50
    m = re.search(r"train(\d+)", run.name)
    run_num = m.group(1) if m else ""
    records.append({"run": run, "run_num": run_num, "best_path": best_path, "map": mAP, "map50": mAP50})

# Pick best (by mAP50-95, then mAP50)
records.sort(key=lambda r: (r["map"], r["map50"]), reverse=True)
winner = records[0]

# Copy + rename
DEST_DIR.mkdir(parents=True, exist_ok=True)
new_name = f"train{winner['run_num']}-{dataset_slug}-best.pt" if winner["run_num"] else f"{winner['run'].name}-{dataset_slug}-best.pt"
out_path = DEST_DIR / new_name
shutil.copy2(winner["best_path"], out_path)

# Summary
print("Evaluated runs:")
for r in records:
    print(f"- {r['run'].name:>8}: mAP50-95={r['map']:.4f} | mAP50={r['map50']:.4f}")
print(f"\nSelected: {winner['run'].name}  →  {out_path}")



## Step 7 — 🔮 Predict (images)

In [ ]:

#@title Upload images and predict
from google.colab import files
from ultralytics import YOLO
from pathlib import Path
import torch

uploaded = files.upload()
img_paths = list(uploaded.keys())
print("Uploaded:", img_paths)

# choose model: latest best or repo weights
RUNS_DIR = Path("/content/YoloV12-Object-Detection-Project/runs/completed-training/")
best_path = None
if RUNS_DIR.exists():
    cands = sorted(RUNS_DIR.glob("**/weights/best.pt"), key=lambda p: p.stat().st_mtime)
    if cands:
        best_path = str(cands[-1])
model_path = best_path if best_path else str(Path("/content/YoloV12-Object-Detection-Project") / "models" / "yolo12s.pt")

model = YOLO(model_path)
pred = model.predict(source=img_paths, imgsz=512, device="cuda" if torch.cuda.is_available() else "cpu", save=True)
print("Saved to:", pred[0].save_dir if isinstance(pred, list) and pred else "Check /content/YoloV12-Object-Detection-Project/runs/completed-predictions")


## Step 8 — 📺 Predict (video)

In [ ]:

#@title Predict on a video (URL or upload)
video_url = ""  #@param {type:"string"}

import urllib.request, os
from google.colab import files
from ultralytics import YOLO
from pathlib import Path
import torch

src = None
if video_url.strip():
    src = "/content/YoloV12-Object-Detection-Project/testing"
    try:
        urllib.request.urlretrieve(video_url, src)
        print("Downloaded:", src)
    except Exception as e:
        print("Failed to download:", e)
        src = None

if not src:
    up = files.upload()
    if up:
        src = list(up.keys())[0]
        print("Uploaded:", src)

assert src, "No video provided."

RUNS_DIR = Path("/content/YoloV12-Object-Detection-Project/runs/completed-training/trainxxx")
best_path = None
if RUNS_DIR.exists():
    cands = sorted(RUNS_DIR.glob("**/weights/best.pt"), key=lambda p: p.stat().st_mtime)
    if cands:
        best_path = str(cands[-1])
model_path = best_path if best_path else str(Path("/content/YoloV12-Object-Detection-Project") / "models" / "yolov12s.pt")

model = YOLO(model_path)
pred = model.predict(source=src, imgsz=512, device="cuda" if torch.cuda.is_available() else "cpu", save=True)
print("Video predictions saved to:", pred[0].save_dir if isinstance(pred, list) and pred else "Check /content/YoloV12-Object-Detection-Project/runs/completed-predictions")


## Step 9 — ♟️ Exporting

Export the directory so you don't lose your folders and work.

In [ ]:
import shutil

# Compress the folder into a ZIP file
shutil.make_archive('runs', 'zip', '/content/runs')
shutil.make_archive('YoloV12-Object-Detection-Project', 'zip', '/content/YoloV12-Object-Detection-Project')

# Download the ZIP file
from google.colab import files
files.download('runs.zip')
files.download('YoloV12-Object-Detection-Project.zip')

In [ ]:
## This will become a script to export as a ready model file

## Troubleshooting


- **Weights YAML missing:** ensure `models/yolov12s.pt` and `yaml/data.yaml` exist in the repo or update paths.
- **No GPU:** Colab may not offer a GPU; the notebook will use CPU (much slower).
- **Val fails:** Train first; then rerun the validate cell.
- **Poor metrics:** Add more data, correct labels, tune `imgsz`/`batch`/`epochs`.
